In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import pickle
from collections import Counter
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from scipy.stats.mstats import gmean
import math
import datetime as dt
import matplotlib
font = {'size': 24}
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
print("Non-first-term-specific models:")
df_new = pd.read_stata("../degree_completion_1/full_data_truncated_enlarged_new.dta")
df_new.loc[:,'available_sum'] = 0
for p in [p for p in list(df_new.columns)[10:] if p.startswith("available") and p != "available_sum"]:
    df_new.loc[:,'available_sum'] += df_new[p]
df_new = df_new[df_new.available_sum > 1]
df_new = df_new.drop(['available_sum'], axis=1)

predictors = pickle.load(open("../degree_completion_1/predictors_rf2.p", "rb"))


l1 = ['coll_lvl_cred_earn', 'prop_comp_pre', 'cum_gpa_pre']
for p in l1:
    v = np.min(df_new[p]) - 1
    df_new.loc[:, p] = df_new.apply(lambda x: v if x['enrolled_pre'] != 1 else x[p], axis=1)
l2 = ['admrate', 'gradrate', 'satvr25', 'satvr75', 'satmt25', 'satmt75', 'satwr25', 'satwr75', 'nsc_coll_type_1', 'nsc_coll_type_2', 'nsc_coll_type_3', 'nsc_coll_type_4', 'nsc_coll_type_5', 'nsc_coll_type_6', 'nsc_coll_type_7', 'nsc_coll_type_8']
for p in l2:
    v = np.min(df_new[p]) - 1
    df_new.loc[:, p] = df_new.apply(lambda x: v if x['enrolled_nsc'] != 1 else x[p], axis=1)
l3 = ['degree_seeking', 'term_cred_att', 'term_gpa', 'prop_comp', 'withdrawn_prop_comp', 'lvl2_prop_comp', 'dev_prop_comp', 'repeat']
for t in ['su', 'fa', 'sp']:
    for i in range(1,7):
        for pp in l3:
            p = pp + "_" + t + str(i)
            v = np.min(df_new[p]) - 1
            df_new.loc[:, p] = df_new.apply(lambda x: v if x['enrolled_' + t + str(i)] != 1 else x[p], axis=1)
l4 = ['enrl_intensity_nsc']
for t in ['su', 'fa', 'sp']:
    for i in range(1,7):
        for pp in l4:
            p = pp + "_" + t + str(i)
            v = np.min(df_new[p]) - 1
            df_new.loc[:, p] = df_new.apply(lambda x: v if x['enrolled_nsc_' + t + str(i)] != 1 else x[p], axis=1)
l5 = ['grants', 'sub_loans', 'unsub_loans', 'others']
for i in range(1,7):
    for pp in l5:
        p = pp + "_yr" + str(i)
        v = np.min(df_new[p]) - 1
        df_new.loc[:, p] = df_new.apply(lambda x: v if x['enrolled_yr' + str(i)] != 1 else x[p], axis=1)
to_drop = ['enrolled_pre', 'enrolled_nsc'] + ['enrolled_' + t + str(i) for t in ['su', 'fa', 'sp'] for i in range(1,7)] + ['enrolled_nsc_' + t + str(i) for t in ['su', 'fa', 'sp'] for i in range(1,7)]
predictors = [p for p in predictors if p not in set(to_drop)]
print(len(predictors))


train_df = df_new[df_new.valid == 0]
test_df = df_new[df_new.valid == 1]


Non-first-term-specific models:
294


In [4]:
train_df.loc[:,['vccsid']].to_stata("train_id_return.dta", write_index=False)
test_df.loc[:,['vccsid']].to_stata("test_id_return.dta", write_index=False)

In [3]:
def calc_cw(y):
    # Calculate the weight of each letter grade to be used in the modeling fitting procedure: the weight is inversely proportional to the square root of the frequency of the letter grade in the training sample
    cw = Counter(y)
    class_weight = {k:np.sqrt(cw.most_common()[0][-1]/v, dtype=np.float32) for k,v in cw.items()}
    return class_weight # The output is a dictionary mapping letter grade to the corresponding weight

In [4]:
optimal_d = 15
optimal_n = 140
optimal_nf = 12
rf = RandomForestClassifier(n_estimators=optimal_n, criterion="entropy",
                            max_depth=optimal_d,
                            random_state=0, n_jobs=-1, max_features=optimal_nf,
                            class_weight = calc_cw(train_df.grad_6years))
rf.fit(train_df.loc[:,predictors], train_df.grad_6years)

RandomForestClassifier(bootstrap=True,
            class_weight={0.0: 1.0, 1.0: 1.2543377}, criterion='entropy',
            max_depth=15, max_features=12, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=140, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [5]:
# Coefficients and predicted scores
y_test_pred_rf = rf.predict_proba(test_df.loc[:,predictors])[:,1]
y_train_pred_rf = rf.predict_proba(train_df.loc[:,predictors])[:,1]
pickle.dump(y_test_pred_rf, open("y_test_pred_rf_return.p", "wb"))
pickle.dump(list(test_df.grad_6years), open( "y_test_return.p", "wb"))
pickle.dump(y_train_pred_rf, open("y_train_pred_rf_return.p", "wb"))
pickle.dump(list(train_df.grad_6years), open("y_train_return.p", "wb"))
print("Random Forest:")
print("Validation AUC = {}".format(round(roc_auc_score(test_df.grad_6years, y_test_pred_rf),4)))
print("Training AUC = {}".format(round(roc_auc_score(train_df.grad_6years, y_train_pred_rf),4)))

Random Forest:
Validation AUC = 0.8932
Training AUC = 0.9248


In [6]:
race_column = []
for i in range(test_df.shape[0]):
    if test_df.white.iloc[i] == 1:
        race_column.append("white")
    elif test_df.afam.iloc[i] == 1:
        race_column.append("afam")
    elif test_df.hisp.iloc[i] == 1:
        race_column.append("hisp")
    elif test_df.asian.iloc[i] == 1:
        race_column.append("asian")
    elif test_df.other.iloc[i] == 1:
        race_column.append("other")
    else:
        race_column.append("mi")
race_column = np.array(race_column)
pred_y = np.array(y_test_pred_rf)
test_y = np.array(test_df.grad_6years)
pred_y = pred_y[race_column != "mi"]
test_y = test_y[race_column != "mi"]
race_column = race_column[race_column != "mi"]
print(len(race_column), len(pred_y), len(test_y))

pred_score_by_race = pd.DataFrame({'race_column': race_column, 'pred_y': pred_y, 'test_y': test_y})
pred_score_by_race.to_csv("pred_score_by_race_return.csv", index=False)

47371 47371 47371
